In [ ]:
import os
from datetime import datetime
from nbconvert import HTMLExporter
from urllib.parse import urljoin
from bs4 import BeautifulSoup

NOTEBOOK = 'weekly_report_2025.ipynb'
OUTDIR = '../../public'

def add_ogp_tags(html_content: str, report_date: datetime, large: bool, description: str) -> str:
    """
    HTMLにOGPメタタグを追加する
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # OGPメタデータの設定
    ogp_data = {
        "og:title": f"ゲームハード週販レポート ({report_date.strftime('%Y.%m.%d')})",
        "og:description": description,
        "og:type": "website",
        "og:url": urljoin("https://gamehardrecord.netlify.app", f"weekly_report_{report_date.strftime('%Y%m%d')}.html")
    }
    
    if large:
        ogp_data["og:image"] = f"https://gamehardrecord.netlify.app/weekly_report_{report_date.strftime('%Y%m%d')}.png"
        twitter_card = "summary_large_image"
    else:
        ogp_data["og:image"] = "https://gamehardrecord.netlify.app/small_card.png"
        twitter_card = "summary"
    
    # headタグを取得
    head = soup.find('head')
    if not head:
        print("Warning: <head> tag not found")
        return html_content
    
    # titleタグを取得（挿入位置の基準）
    title_tag = head.find('title')
    
    # OGPタグを追加（titleの前に挿入し、各タグ後に改行）
    for property_name, content in ogp_data.items():
        meta_tag = soup.new_tag('meta')
        meta_tag['property'] = property_name
        meta_tag['content'] = content
        if title_tag:
            title_tag.insert_before(meta_tag)
            meta_tag.insert_after(soup.new_string('\n'))
        else:
            head.insert(0, meta_tag)
            meta_tag.insert_after(soup.new_string('\n'))
    
    # Twitter Cardタグを追加（titleの前に挿入し、各タグ後に改行）
    twitter_tags = {
        "twitter:card": twitter_card,
        "twitter:title": ogp_data["og:title"],
        "twitter:description": ogp_data["og:description"],
        "twitter:image": ogp_data["og:image"]
    }
    
    for name, content in twitter_tags.items():
        meta_tag = soup.new_tag('meta')
        meta_tag['name'] = name
        meta_tag['content'] = content
        if title_tag:
            title_tag.insert_before(meta_tag)
            meta_tag.insert_after(soup.new_string('\n'))
        else:
            head.insert(0, meta_tag)
            meta_tag.insert_after(soup.new_string('\n'))
    
    print(f"✓ Added {len(ogp_data) + len(twitter_tags)} meta tags")
    return str(soup)


from report_config import get_config
config = get_config()
report_date = config["date"]
report_file_name = f"weekly_report_{report_date.strftime('%Y%m%d')}.html"
report_path = f"{OUTDIR}/{report_file_name}"

# report_pathにファイルが存在したら削除
if os.path.exists(report_path):
    os.remove(report_path)

# nbconvertでHTMLに変換（標準テンプレートを使用）
exporter = HTMLExporter()
exporter.template_name = 'lab'  # 標準テンプレートを使用
exporter.exclude_input = True  # 入力セルを除外

try:
    print("Converting notebook to HTML...")
    body, resources = exporter.from_filename(NOTEBOOK, resources={})
    
    print("Adding OGP tags to HTML...")
    # OGPタグを追加
    modified_html = add_ogp_tags(body, config["date"], config["large"], config["description"])
    
    # ファイルに保存
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(modified_html)
    print("Report published successfully.")
    
    # HTMLファイルにOGPタグが含まれているか確認
    if 'og:title' in modified_html:
        print("✓ OGP tags found in HTML")
        
        # タイトルも確認
        soup = BeautifulSoup(modified_html, 'html.parser')
        title_tag = soup.find('meta', {'property': 'og:title'})
        if title_tag:
            print(f"✓ OGP title: {title_tag.get('content')}")
    else:
        print("✗ OGP tags not found in HTML")
        
except Exception as e:
    print(f"Error during export: {e}")
    import traceback
    traceback.print_exc()

Converting notebook to HTML...
Adding OGP tags to HTML...
✓ Added 9 meta tags
Report published successfully.
✓ OGP tags found in HTML
✓ OGP title: ゲームハード週販レポート (2025.08.31)
Adding OGP tags to HTML...
✓ Added 9 meta tags
Report published successfully.
✓ OGP tags found in HTML
✓ OGP title: ゲームハード週販レポート (2025.08.31)
